In [6]:

import os
from timeit import time
import warnings
import sys
import cv2
import numpy as np
from PIL import Image

from deep_sort.application_util import preprocessing
from deep_sort.deep_sort import nn_matching
from deep_sort.deep_sort.detection import Detection
from deep_sort.deep_sort.tracker import Tracker
from deep_sort.tools import generate_detections as gdet
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image,resize_image
from deep_sort.detection import Detection as ddet
warnings.filterwarnings('ignore')


/home/bhuwan/anaconda3/envs/torch/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/bhuwan/anaconda3/envs/torch/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/bhuwan/anaconda3/envs/torch/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/bhuwan/anaconda3/envs/torch/lib/python3.7/s

ModuleNotFoundError: No module named 'deep_sort.detection'

In [ ]:

def pedestrian_tracking(model):

   # Definition of the parameters
    max_cosine_distance = 0.3
    nn_budget = None
    nms_max_overlap = 1.0
    
   # deep_sort 
    model_filename = 'mars-small128.pb'
    encoder = gdet.create_box_encoder(model_filename,batch_size=1)
    
    metric = nn_matching.NearestNeighborDistanceMetric("cosine", max_cosine_distance, nn_budget)
    tracker = Tracker(metric)

    writeVideo_flag = True 
    
    video_capture = cv2.VideoCapture('/home/bhuwan/Downloads/Tuomiokirkko.mp4')
   
    labels_to_names = {0: 'person'}

    if writeVideo_flag:
    # Define the codec and create VideoWriter object
        w = int(video_capture.get(3))
        h = int(video_capture.get(4))
        fourcc = cv2.VideoWriter_fourcc(*'MJPG')
        out = cv2.VideoWriter('num8_2_v6.mp4', fourcc, 30, (w, h))
        list_file = open('detection.txt', 'w')
        frame_index = -1 
        
    fps = 0.0
    i=0;
    while True:
        ret, frame = video_capture.read()  
        if ret != True:
            break;
        t1 = time.time()
        
        image = preprocess_image(frame)
        boxes, scores, labels = model.predict_on_batch(np.expand_dims(image,axis=0))
        i = 0
        box_tmp = []
        for box,label,score in zip(boxes[0], labels[0], scores[0]):
            if np.array(score) < 0.15:
                break
            
            box_tmp.append((box[0:2]).tolist()+(box[2:4]-box[0:2]).tolist())
            i += 1
        # box_tmp = boxes[0][0:i]

        # box_tmp = box_tmp.tolist()
       # print("box_num",len(boxs))
        features = encoder(frame,box_tmp)
        
        # score to 1.0 here).
        detections = [Detection(bbox, 1.0, feature) for bbox, feature in zip(box_tmp, features)]
        
        # Run non-maxima suppression.
        boxes = np.array([d.tlwh for d in detections])
        scores = np.array([d.confidence for d in detections])
        indices = preprocessing.non_max_suppression(np.array(box_tmp), nms_max_overlap, scores)
        detections = [detections[i] for i in indices]
        # print("indices:",indices)
        # print("detection: ",detections[indices[0]]);
        # Call the tracker
        tracker.predict()
        tracker.update(detections)
        
        fps  = ( fps + (1./(time.time()-t1)) ) / 2
        #print("fps= %f"%(fps))  

        for track in tracker.tracks:
            if not track.is_confirmed() or track.time_since_update > 1:
                continue 
            bbox = track.to_tlbr()
            cv2.rectangle(frame, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])),(255,255,255), 2)
            cv2.putText(frame, str(track.track_id),(int(bbox[0]), int(bbox[1])),0, 5e-3 * 200, (0,255,0),2)


        for j,det in enumerate(detections):
            bbox = det.to_tlbr()
            cv2.rectangle(frame,(int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])),(255,0,0), 2)
            # cv2.putText(frame,str(labels_to_names[j]),(int(bbox[0]),int(bbox[1])-35),0,5e-3*200,(143,17,86),2)
        cv2.namedWindow("track result", 0)
        cv2.resizeWindow("track result", 1280, 720)  
        cv2.imshow('track result', frame)
        
        if writeVideo_flag:
            # save a frame
            out.write(frame)
            frame_index = frame_index + 1
            list_file.write(str(frame_index)+' ')
            if len(boxes) != 0:
                for i in range(0,len(boxes)):
                    list_file.write(str(boxes[i][0]) + ' '+str(boxes[i][1]) + ' '+str(boxes[i][2]) + ' '+str(boxes[i][3]) + ' ')
            list_file.write('\n')
            
       
        
        # Press Q to stop!
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    video_capture.release()
    if writeVideo_flag:
        out.release()
        list_file.close()
    cv2.destroyAllWindows()




In [3]:

model_path = '/home/bhuwan/Desktop/video/model.h5'
model = models.load_model(model_path,backbone_name='resnet50')
pedestrian_tracking(model)


tracking <tf.Variable 'Variable:0' shape=(15, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_1:0' shape=(15, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_2:0' shape=(15, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_3:0' shape=(15, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_4:0' shape=(15, 4) dtype=float32> anchors



Using TensorFlow backend.



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where





